In [2]:
import pandas as pd
from pulp import *

In [4]:
d = {'Supply_Region':['USA', 'Germany', 'Japan', 'Brazil', 'India'], 'Dmd':[2719.6,84.1,1676.8,145.4,156.4]}
v = {'Supply_Region':['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'USA':[6,13,20,12,22],'Germany':[13,6,14,14,13],'Japan':[20,14,3,21,10], 
    'Brazil':[12,14,21,8,23], 'India':[17,13,9,21,8]}
f = {'Supply_Region':['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'Low_Cap':[6500,4980,6230,3230,2110], 'High_Cap':[9500,7270,9100,4730,3080]}
p = {'Supply_Region':['USA', 'Germany', 'Japan', 'Brazil', 'India'],
    'Low_Cap':[500,500,500,500,500], 'High_Cap':[1500,1500,1500,1500,1500]}

demand = pd.DataFrame(data = d)
demand = demand.set_index('Supply_Region')

var_cost = pd.DataFrame(data = v)
var_cost = var_cost.set_index('Supply_Region')

fix_cost = pd.DataFrame(data = f)
fix_cost = fix_cost.set_index('Supply_Region')

cap = pd.DataFrame(data = p)
cap = cap.set_index('Supply_Region')

print(fix_cost)
print(cap)
print(demand)

               Low_Cap  High_Cap
Supply_Region                   
USA               6500      9500
Germany           4980      7270
Japan             6230      9100
Brazil            3230      4730
India             2110      3080
               Low_Cap  High_Cap
Supply_Region                   
USA                500      1500
Germany            500      1500
Japan              500      1500
Brazil             500      1500
India              500      1500
                  Dmd
Supply_Region        
USA            2719.6
Germany          84.1
Japan          1676.8
Brazil          145.4
India           156.4


In [7]:
# Initialize, Define Decision Vars., and Objective Function
model = LpProblem("Capacitated Plant Location Model", LpMinimize)
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low_Cap','High_Cap']
x = LpVariable.dicts("production", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts("plant", 
                     [(i,s) for s in size for i in loc], cat='Binary')


In [8]:
model += (lpSum([ fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc ])
          + lpSum([ var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc ]))

# Define the constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Dmd']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[i,s] 
                                                       for s in size])

In [11]:
model.solve()
print(LpStatus[model.status])
o = [{'prod':'{} to {}'.format(i,j),'quant':x[(i,j)].varValue} for i in loc for j in loc]
print(pd.DataFrame(o))

o = [{'loc':i, 'lc':y[(i,size[0])].varValue, 'hc':y[(i,size[1])].varValue} for i in loc]
print(pd.DataFrame(o))



print("Objective = $", value(model.objective))

o = [{'name':name, 'shadow_price':c.pi, 'slack':c.slack} for name, c in model.constraints.items()]
print(pd.DataFrame(o))


Optimal
                  prod   quant
0           USA to USA  1500.0
1       USA to Germany     0.0
2         USA to Japan     0.0
3        USA to Brazil     0.0
4         USA to India     0.0
5       Germany to USA     0.0
6   Germany to Germany     0.0
7     Germany to Japan     0.0
8    Germany to Brazil     0.0
9     Germany to India     0.0
10        Japan to USA     0.0
11    Japan to Germany     0.0
12      Japan to Japan  1500.0
13     Japan to Brazil     0.0
14      Japan to India     0.0
15       Brazil to USA  1219.6
16   Brazil to Germany     0.0
17     Brazil to Japan     0.0
18    Brazil to Brazil   145.4
19     Brazil to India     0.0
20        India to USA     0.0
21    India to Germany    84.1
22      India to Japan   176.8
23     India to Brazil     0.0
24      India to India   156.4
    hc   lc      loc
0  1.0  0.0      USA
1  0.0  0.0  Germany
2  1.0  0.0    Japan
3  1.0  0.0   Brazil
4  0.0  1.0    India
Objective = $ 58850.899999999994
   name  shadow_price  slac

In [10]:
model

Capacitated Plant Location Model:
MINIMIZE
4730*plant_('Brazil',_'High_Cap') + 3230*plant_('Brazil',_'Low_Cap') + 7270*plant_('Germany',_'High_Cap') + 4980*plant_('Germany',_'Low_Cap') + 3080*plant_('India',_'High_Cap') + 2110*plant_('India',_'Low_Cap') + 9100*plant_('Japan',_'High_Cap') + 6230*plant_('Japan',_'Low_Cap') + 9500*plant_('USA',_'High_Cap') + 6500*plant_('USA',_'Low_Cap') + 8*production_('Brazil',_'Brazil') + 14*production_('Brazil',_'Germany') + 21*production_('Brazil',_'India') + 21*production_('Brazil',_'Japan') + 12*production_('Brazil',_'USA') + 14*production_('Germany',_'Brazil') + 6*production_('Germany',_'Germany') + 13*production_('Germany',_'India') + 14*production_('Germany',_'Japan') + 13*production_('Germany',_'USA') + 23*production_('India',_'Brazil') + 13*production_('India',_'Germany') + 8*production_('India',_'India') + 10*production_('India',_'Japan') + 22*production_('India',_'USA') + 21*production_('Japan',_'Brazil') + 14*production_('Japan',_'Germany')